In [20]:
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import unidecode
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep
import time
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

# **Web scraping** Kaspi.kz

In [13]:
driver = webdriver.Chrome()
url= 'https://kaspi.kz/shop/c/smartphones/'
driver.get(url)
#open the kaspi kz(smartphones)

In [15]:
brands = [
    "CORN","Blackview", "Apple", "ASUS",  "BQ", "Caterpillar", "Coolpad",  "CUBOT", 
    "DOOGEE", "F+", "FOSSiBOT", "Frbby", "Google", "HONOR", "HOTWAV", "Huawei", 
    "IIIF150", "Infinix", "INOI", "iQOO", "Itel", "IXXI", "Lenovo", "Lingbo", 
    "MAXVI", "Meizu", "M-HORSE", "Motorola", "Nokia", "NOTHING", "Nubia", 
    "OnePlus", "OPPO", "OSCAL", "Oukitel", "Poco", "Realme", "Redmi", 
    "Samsung", "Sony", "SOYES", "STARLET", "TECNO", "Ulefone", "UMIDIGI", 
    "Umiio", "Unihertz", "Vivo", "W&O", "WIKO", "Xiaomi", "ZTE"
]
#all smartphones brands from kaspi

In [16]:
#parsing
all_product_links = []
current_links = []
wait = WebDriverWait(driver, 2)

#to brands not repeat
selected_brands = []

for brand in brands:
    # check the brand
    if brand in selected_brands:
        continue 

    try:
        #shom more button to see all the brands
        show_more_button = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//span[contains(@class, 'filters__spoiler') and contains(text(), 'Показать еще')]"
        )))
        show_more_button.click() 
        time.sleep(2)  
    except Exception as e:
        print("Shom more doesn't exist")

    try:
        filter_label = wait.until(EC.element_to_be_clickable((
            By.XPATH, f"//span[contains(text(), '{brand}')]"
        ))) 
        filter_label.click()  # choose the brand 
        time.sleep(5)  #wait to load the page
        print(f"Filter {brand} apllied")
    except Exception as e:
        print(f"Error for the {brand}.")

    try:
        # collect the links
        while True:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            elements = soup.findAll('a', class_="item-card__name-link")
    
            for j in elements:
                link = j.get('href')
                current_links.append(link)

            try:
                next_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.XPATH, '//li[@class="pagination__el" and contains(text(), "Следующая →")]'))
                ) #next button for the next page
                driver.execute_script('arguments[0].scrollIntoView();', next_button)
                next_button.click()
                
            except Exception as e:
                break
            time.sleep(6)  # wait to load the page
    except Exception as e:
        print(f"Error {brand}.")

    all_product_links.extend(current_links) #extend to the all links
    current_links = []  # clean the current link

    # reset the filter with click the button again
    try:
        filter_label = wait.until(EC.element_to_be_clickable((
            By.XPATH, f"//span[contains(text(), '{brand}')]"
        )))
        filter_label.click()
        time.sleep(5)
        print(f"Filter reset for {brand}.")
    except Exception as e:
        print(f"Can't reset {brand}.")

    selected_brands.append(brand) #append the brand for the selected

    time.sleep(5) 

print(f"All links len it's : {len(all_product_links)}")



Фильтр для бренда CORN применен.
Фильтр сброшен для бренда CORN.
Фильтр для бренда Blackview применен.
Фильтр сброшен для бренда Blackview.
Фильтр для бренда Apple применен.
Фильтр сброшен для бренда Apple.
Фильтр для бренда ASUS применен.
Фильтр сброшен для бренда ASUS.
Фильтр для бренда BQ применен.
Фильтр сброшен для бренда BQ.
Фильтр для бренда Caterpillar применен.
Фильтр сброшен для бренда Caterpillar.
Фильтр для бренда Coolpad применен.
Фильтр сброшен для бренда Coolpad.
Фильтр для бренда CUBOT применен.
Фильтр сброшен для бренда CUBOT.
Фильтр для бренда DOOGEE применен.
Фильтр сброшен для бренда DOOGEE.
Фильтр для бренда F+ применен.
Фильтр сброшен для бренда F+.
Фильтр для бренда FOSSiBOT применен.
Фильтр сброшен для бренда FOSSiBOT.
Фильтр для бренда Frbby применен.
Фильтр сброшен для бренда Frbby.
Фильтр для бренда Google применен.
Фильтр сброшен для бренда Google.
Фильтр для бренда HONOR применен.
Фильтр сброшен для бренда HONOR.
Фильтр для бренда HOTWAV применен.
Фильтр сб

In [17]:
len(all_product_links)

4920

In [18]:
with open('links_new.txt', 'w') as file:
    for link in all_product_links:
        file.write(link + '\n') 
        
print("All links in 'links_new.txt'")
#save the file

All links in 'links_new.txt'


**Divided it for 8 parts and take the specifications separately**

In [ ]:
#divided in the 8 groups
chunk_size = len(all_product_links) // 8  
chunks = [all_product_links[i:i + chunk_size] for i in range(0, len(all_product_links), chunk_size)]

if len(all_product_links) % 8 != 0:
    chunks[-2].extend(chunks[-1])  
    chunks = chunks[:-1]  


for i, chunk in enumerate(chunks):
    filename = f'links_part_{i+1}.txt'
    with open(filename, 'w', encoding='utf-8') as f:
        for link in chunk:
            f.write(link + '\n')
    print(f'Часть {i+1} сохранена в {filename}')


In [ ]:
#example of the first part
with open('links_part_1.txt', 'r', encoding='utf-8') as file:
    all_product_links1 = [line.strip() for line in file.readlines()] #open the links_part_1


In [ ]:

df = pd.DataFrame(columns=['name', 'price', 'NFC', 'Color', 'Тип экрана', 'Диагональ', 'RAM', 'Память', 'Емкость акк'])
#create a dataframe

options = Options()
options.headless = True  
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#driver setting
start_time = time.time()
#time
counter = 0

for link in all_product_links1:
    counter += 1  
    print(f"Collecting {counter}/{len(all_product_links1)}: {link}")

    
    driver.get("https://kaspi.kz" + link)
    
    # close the 'choose a city page'
    try:
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'dialog__close'))
        )
        close_button.click()
    except:
        print('collect')
    
    
    time.sleep(5)

    #parser
    page_source = driver.page_source

   
    soup = BeautifulSoup(page_source, 'html.parser')

   #parsing the name
    try:
        name = soup.find('h1', class_="item__heading").text.strip()
    except:
        name = np.nan 
#collecting the price
    try:
        price = soup.find('div', class_="item__price-once").text.strip()
    except:
        price = np.nan
    
     #specifications set to the nan
    nfc_technology = np.nan
    color = np.nan
    screen_type = np.nan
    screen_size = np.nan
    ram_size = np.nan
    internal_storage = np.nan
    battery_capacity = np.nan

    try:
        #we have the same class for the specifications
        specs = soup.find_all('li', class_='short-specifications__text')
        #so take it with if elif
        for spec in specs:
            spec_text = spec.text.strip()  
            
            if 'Технология NFC' in spec_text:
                nfc_technology = spec_text.split(":")[1].strip()
            elif 'Цвет' in spec_text:
                color = spec_text.split(":")[1].strip()
            elif 'Тип экрана' in spec_text:
                screen_type = spec_text.split(":")[1].strip()
            elif 'Диагональ' in spec_text:
                screen_size = spec_text.split(":")[1].strip()
            elif 'Размер оперативной памяти' in spec_text:
                ram_size = spec_text.split(":")[1].strip()
            elif 'Объем встроенной памяти' in spec_text:
                internal_storage = spec_text.split(":")[1].strip()
            elif 'Емкость аккумулятора' in spec_text:
                battery_capacity = spec_text.split(":")[1].strip()

    except Exception as e:
        print(f"Error {link}: {e}")
    
    # row for the dataframe
    row = [name, price, nfc_technology, color, screen_type, screen_size, ram_size, internal_storage, battery_capacity]
    
    # connect
    df.loc[len(df)] = row

# end time #time take the 9600 sec
end_time = time.time()


# close the browser
driver.quit()


In [ ]:
df.to_csv('part1.csv', index=False, encoding='utf-8')
#save the data

#### **And so it is for the other 7 parts**

## Connect the all data

In [4]:
df2 = pd.read_csv('part2.csv')
df3 = pd.read_csv('part3.csv')
df4 = pd.read_csv('part4.csv')
df5 = pd.read_csv('part5.csv')
df7 = pd.read_csv('part7.csv')
df8 = pd.read_csv('part8.csv')

**In the parts 1 and 6, prices don't takes, so i run it again in the data spell and save it**

In [5]:
price4 = pd.read_csv('C:/Users/Niyaz/Downloads/part4.csv') #i just mix up
price1 = pd.read_csv('C:/Users/Niyaz/Downloads/part1.csv')

In [6]:
df1 = pd.read_csv('part1not.csv')
df6 = pd.read_csv('part6not.csv') #the dataframes without prices

In [7]:
df1['price'] = price1['price']
df6['price'] = price4['price']
#replace the prices

In [9]:
df1.head()

,name,price,NFC,Color,Тип экрана,Диагональ,RAM,Память,Емкость акк
0,Смартфон CORN C55 Pro 2 ГБ/64 ГБ черный,78 990 ₸,Нет,черный,"IPS, сенсорный",5.5 дюйм,2 ГБ,64.0 ГБ,2600.0 мАч
1,Смартфон CORN C55 Pro 2 ГБ/64 ГБ синий,79 000 ₸,Нет,синий,"IPS, сенсорный",5.5 дюйм,2 ГБ,64.0 ГБ,2600.0 мАч
2,Смартфон CORN X50 2 ГБ/16 ГБ черный,47 990 ₸,Нет,черный,"IPS, сенсорный",5.0 дюйм,2 ГБ,16.0 ГБ,2000.0 мАч
3,Смартфон CORN X50 2 ГБ/16 ГБ синий,53 550 ₸,Нет,синий,"IPS, сенсорный",5.0 дюйм,2 ГБ,16.0 ГБ,2000.0 мАч
4,Смартфон Blackview BV8900 8 ГБ/256 ГБ оранжевый,129 990 ₸,Да,оранжевый,"IPS, сенсорный, Corning Gorilla Glass 7",6.5 дюйм,8 ГБ,256.0 ГБ,10000.0 мАч


In [10]:
df6.head()

,name,price,NFC,Color,Тип экрана,Диагональ,RAM,Память,Емкость акк
0,Смартфон OnePlus Ace 8 ГБ/256 ГБ синий,229 990 ₸,Да,синий,"6.7"" (1080x2412), OLED",6.7 дюйм,8 ГБ,256.0 ГБ,4500.0 мАч
1,Смартфон OnePlus 9 Pro 8 ГБ/128 ГБ черный,319 990 ₸,Да,черный,"цветной AMOLED, сенсорный",6.7 дюйм,8 ГБ,128.0 ГБ,4500.0 мАч
2,Смартфон OnePlus ACE 2 16 ГБ/512 ГБ черный,399 990 ₸,Да,черный,"AMOLED, сенсорный, Corning Gorilla Glass 5",6.74 дюйм,16 ГБ,512.0 ГБ,5000.0 мАч
3,Смартфон OnePlus Ace 2 Genshin Impact Edition ...,599 990 ₸,Да,красный,"AMOLED, сенсорный, Corning Gorilla Glass 5",6.74 дюйм,18 ГБ,512.0 ГБ,5000.0 мАч
4,Смартфон OPPO A78 8 ГБ/256 ГБ черный,77 923 ₸,Да,черный,"AMOLED, сенсорный, Corning Gorilla Glass 5",6.43 дюйм,8 ГБ,256.0 ГБ,5000.0 мАч


In [8]:
all_df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8], ignore_index=True)#concat

In [11]:
all_df

,name,price,NFC,Color,Тип экрана,Диагональ,RAM,Память,Емкость акк
0,Смартфон CORN C55 Pro 2 ГБ/64 ГБ черный,78 990 ₸,Нет,черный,"IPS, сенсорный",5.5 дюйм,2 ГБ,64.0 ГБ,2600.0 мАч
1,Смартфон CORN C55 Pro 2 ГБ/64 ГБ синий,79 000 ₸,Нет,синий,"IPS, сенсорный",5.5 дюйм,2 ГБ,64.0 ГБ,2600.0 мАч
2,Смартфон CORN X50 2 ГБ/16 ГБ черный,47 990 ₸,Нет,черный,"IPS, сенсорный",5.0 дюйм,2 ГБ,16.0 ГБ,2000.0 мАч
3,Смартфон CORN X50 2 ГБ/16 ГБ синий,53 550 ₸,Нет,синий,"IPS, сенсорный",5.0 дюйм,2 ГБ,16.0 ГБ,2000.0 мАч
4,Смартфон Blackview BV8900 8 ГБ/256 ГБ оранжевый,129 990 ₸,Да,оранжевый,"IPS, сенсорный, Corning Gorilla Glass 7",6.5 дюйм,8 ГБ,256.0 ГБ,10000.0 мАч
...,...,...,...,...,...,...,...,...,...
4915,Смартфон Coolpad CP12S 8 ГБ/256 ГБ черный,99 000 ₸,Да,черный,"IPS, сенсорный",6.79 дюйм,8 ГБ,256.0 ГБ,5000.0 мАч
4916,Смартфон Coolpad CP12S 6 ГБ/128 ГБ зеленый,79 000 ₸,Да,зеленый,"IPS, сенсорный",6.79 дюйм,6 ГБ,128.0 ГБ,5000.0 мАч
4917,Смартфон Coolpad CP12 4 ГБ/64 ГБ синий,54 900 ₸,Нет,синий,IPS LCD,6.517 дюйм,4 ГБ,64.0 ГБ,4500.0 мАч
4918,Смартфон Coolpad CP12 4 ГБ/128 ГБ синий,59 000 ₸,Нет,синий,IPS LCD,6.517 дюйм,4 ГБ,128.0 ГБ,4500.0 мАч


In [12]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         4920 non-null   object
 1   price        4920 non-null   object
 2   NFC          4920 non-null   object
 3   Color        4920 non-null   object
 4   Тип экрана   4920 non-null   object
 5   Диагональ    4920 non-null   object
 6   RAM          4920 non-null   object
 7   Память       4920 non-null   object
 8   Емкость акк  4920 non-null   object
dtypes: object(9)
memory usage: 346.1+ KB


In [ ]:
#all_df.to_csv('collect_phonetype1.csv') #save the data

### **Also i take the rating and the reviews count in the data spell**

In [13]:
#code for this
df = pd.DataFrame(columns=['rating', 'reviews'])

counter = 0
driver = webdriver.Chrome()
for link in all_product_links:
    counter +=1
    driver.get("https://kaspi.kz" + link)
    
    try:
        close_button = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'dialog__close'))
        )
        close_button.click()
    except:
        print('collect')
    page_source = driver.page_source

   
    soup = BeautifulSoup(page_source, 'html.parser')

    #collect the rating and the reviews
    try:
        rating_class = soup.find('span', class_='rating')['class'][1]  
        rating = float(rating_class[1:]) / 10  
    except (AttributeError, IndexError, ValueError):
        rating = np.nan  
   
    try:
        reviews_text = soup.find('a', class_='item__rating-link').find('span').text
        reviews = int(reviews_text.split()[0][1:-1])  # Извлекаем число из текста "(1613 отзывов)"
    except (AttributeError, ValueError):
        reviews = np.nan  


    row = [rating,reviews]

    df.loc[len(df)] = row




driver.quit()


In [14]:
rate = pd.read_csv('C:/Users/Niyaz/Downloads/rate.csv')

In [18]:
rate

,Unnamed: 0,rating,reviews
0,0,0.0,NaN
1,1,0.0,NaN
2,2,0.0,NaN
3,3,0.0,NaN
4,4,4.9,NaN
...,...,...,...
4915,4915,5.0,NaN
4916,4916,4.7,NaN
4917,4917,0.0,NaN
4918,4918,0.0,NaN


In [21]:
main_df = all_df.reset_index(drop=True)
rating_df = rate.reset_index(drop=True)

In [22]:
all_data = pd.concat([main_df, rating_df[['rating', 'reviews']]], axis=1)

In [23]:
all_data

,name,price,NFC,Color,Тип экрана,Диагональ,RAM,Память,Емкость акк,rating,reviews
0,Смартфон CORN C55 Pro 2 ГБ/64 ГБ черный,78 990 ₸,Нет,черный,"IPS, сенсорный",5.5 дюйм,2 ГБ,64.0 ГБ,2600.0 мАч,0.0,NaN
1,Смартфон CORN C55 Pro 2 ГБ/64 ГБ синий,79 000 ₸,Нет,синий,"IPS, сенсорный",5.5 дюйм,2 ГБ,64.0 ГБ,2600.0 мАч,0.0,NaN
2,Смартфон CORN X50 2 ГБ/16 ГБ черный,47 990 ₸,Нет,черный,"IPS, сенсорный",5.0 дюйм,2 ГБ,16.0 ГБ,2000.0 мАч,0.0,NaN
3,Смартфон CORN X50 2 ГБ/16 ГБ синий,53 550 ₸,Нет,синий,"IPS, сенсорный",5.0 дюйм,2 ГБ,16.0 ГБ,2000.0 мАч,0.0,NaN
4,Смартфон Blackview BV8900 8 ГБ/256 ГБ оранжевый,129 990 ₸,Да,оранжевый,"IPS, сенсорный, Corning Gorilla Glass 7",6.5 дюйм,8 ГБ,256.0 ГБ,10000.0 мАч,4.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4915,Смартфон Coolpad CP12S 8 ГБ/256 ГБ черный,99 000 ₸,Да,черный,"IPS, сенсорный",6.79 дюйм,8 ГБ,256.0 ГБ,5000.0 мАч,5.0,NaN
4916,Смартфон Coolpad CP12S 6 ГБ/128 ГБ зеленый,79 000 ₸,Да,зеленый,"IPS, сенсорный",6.79 дюйм,6 ГБ,128.0 ГБ,5000.0 мАч,4.7,NaN
4917,Смартфон Coolpad CP12 4 ГБ/64 ГБ синий,54 900 ₸,Нет,синий,IPS LCD,6.517 дюйм,4 ГБ,64.0 ГБ,4500.0 мАч,0.0,NaN
4918,Смартфон Coolpad CP12 4 ГБ/128 ГБ синий,59 000 ₸,Нет,синий,IPS LCD,6.517 дюйм,4 ГБ,128.0 ГБ,4500.0 мАч,0.0,NaN


In [24]:
all_data.to_csv('all_data.csv')